In [17]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import optim

In [18]:
import sys
sys.path.insert(1, "../../")

In [19]:
from train import *
from data_preprocessing import *
from Models.darknet19 import *

In [20]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu') 

In [21]:
model = Darknet19(num_classes=10, device=device, dtype=torch.float32)

In [22]:
data_path = '../../../dummy_datasets/'
norms_path = os.path.join(data_path, 'norms.json')

In [23]:
means = get_means(path=norms_path, train_loader=None)
stds = get_stds(path=norms_path, train_loader=None)

Means are: [0.4405549168586731, 0.4407285749912262, 0.4381718039512634]
stds are: [0.25142669677734375, 0.25270089507102966, 0.25131651759147644]


In [24]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((224, 224)),
                                      transforms.Normalize(mean=means, std=stds)])

In [25]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)
val_dataset = ImageNetSubset(path=data_path, train=False, transform=transformations, half=False, show=False)

In [26]:
epochs=10
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=4)

In [27]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)

In [28]:
loss_fn = nn.NLLLoss()

In [29]:
prof = torch.profiler.profile(
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=epochs-1, repeat=1),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('../log/darknet19/training'),
        record_shapes=True,
        profile_memory=True,
        with_flops=True,
        with_modules=True,
        with_stack=True)


Profiling your personal module 
https://pytorch.org/tutorials/beginner/profiler.html

In [30]:
history, gradient_stats, prof = train(epochs, train_loader, val_loader, model, optimizer, loss_fn, prof, model_path = '../log/darknet19/training/models/')

2024-11-20 21:03:12.683832 Epoch 1 
2024-11-20 21:03:20.503767 Batch 1 
[Train] Accuracy: 10.0%, Loss per batch: 2.3006
2024-11-20 21:03:31.985205 Batch 1 
[Val] Accuracy: 10.0%, loss per batch: 2.3026
2024-11-20 21:03:35.002479 Epoch 2 
2024-11-20 21:03:42.194661 Batch 1 
[Train] Accuracy: 40.0%, Loss per batch: 2.0571
2024-11-20 21:03:54.508004 Batch 1 
[Val] Accuracy: 10.0%, loss per batch: 2.3027
2024-11-20 21:03:57.794698 Epoch 3 
2024-11-20 21:04:04.667948 Batch 1 
[Train] Accuracy: 76.6667%, Loss per batch: 1.8533
2024-11-20 21:04:16.021233 Batch 1 
[Val] Accuracy: 10.0%, loss per batch: 2.3029
2024-11-20 21:04:19.209112 Epoch 4 
2024-11-20 21:04:26.609973 Batch 1 
[Train] Accuracy: 86.6667%, Loss per batch: 1.5991
2024-11-20 21:04:39.518533 Batch 1 
[Val] Accuracy: 10.0%, loss per batch: 2.3035
2024-11-20 21:04:42.690746 Epoch 5 
2024-11-20 21:04:50.500902 Batch 1 
[Train] Accuracy: 93.3333%, Loss per batch: 1.3299
2024-11-20 21:05:03.106687 Batch 1 
[Val] Accuracy: 10.0%, loss

In [31]:
print(prof.key_averages().table(row_limit=-1))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  Total MFLOPs  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         0.63%        1.383s       100.00%      218.369s       24.263s      73.62 Mb      -1.99 Gb             9            --  
                                          Training Loop         0.02%      50.267ms        51.74%      112.981s       12.553s      73.60 Mb    -155.05 Mb             9            --  
                     Validation Loop Before Data Loader         4.08%        8.9

In [32]:
save_train_outputs('../log/darknet19/training', history, gradient_stats)